                        Project done by Michela Pirozzi MAT:732531 and Sara Ferioli MAT:733105

# LAB 2

## Goal

In this lab, the goal will be to prepare data and then apply it to the model. 

### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import math
from sklearn import preprocessing
from scipy.stats import shapiro
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

### Functions

In [2]:
# Method that allow to load ateco csv and prepare the data for the join
def addingAteco():
    df_ateco = pd.read_csv("ateco.csv")
    df_ateco.Descrizione_ateco.replace(r'^.* - ' , '',regex=True, inplace = True,)
    df_ateco = df_ateco.drop("_id",axis=1)
    df_ateco.rename(columns = {"Descrizione_ateco":"SETTOREECONOMICODETTAGLIO"}, inplace = True)
    df_ateco['SETTOREECONOMICODETTAGLIO'] = df_ateco['SETTOREECONOMICODETTAGLIO'].str.upper()
    df_ateco.drop_duplicates(subset="SETTOREECONOMICODETTAGLIO",keep="first",inplace=True)
    return df_ateco

### Import data and analysis

In [3]:
# Load the dataframe
attivati = pd.read_csv("Rapporti_di_lavoro_attivati.csv")

In [4]:
ateco = addingAteco()
ateco

,Codice_ateco,SETTOREECONOMICODETTAGLIO
0,01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
1,01.1,COLTIVAZIONE DI COLTURE AGRICOLE NON PERMANENTI
2,01.11,"COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO), LEG..."
3,01.11.1,COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO)
5,01.11.2,COLTIVAZIONE DI SEMI OLEOSI
...,...,...
3100,96.09.09,ALTRE ATTIVITÀ DI SERVIZI PER LA PERSONA NCA
3101,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3106,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...
3107,98.1,PRODUZIONE DI BENI INDIFFERENZIATI PER USO PRO...


In [5]:
# Transformations of the strings in upper case, to make join
attivati['SETTOREECONOMICODETTAGLIO'] = attivati['SETTOREECONOMICODETTAGLIO'].str.upper()

In [6]:
# Retrieve only the year from the date 
attivati['DATA'] = pd.to_datetime(attivati['DATA'])
attivati ['ANNO'] = attivati.DATA.dt.year
attivati = attivati.drop(labels='DATA', axis=1)

In [7]:
# Join between the two DataFrames
join = pd.merge(attivati, ateco, on =['SETTOREECONOMICODETTAGLIO'], how='inner')
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97
...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020,49.50.2
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021,93.19.91
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012,93.19.91
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018,93.19.91


In [8]:
# Obtain only the first two digits of the 'codice ateco' to understand the macro category
join.Codice_ateco.replace(r'^(..)\..*' , r'\1',regex=True, inplace = True,)
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97
...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020,49
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021,93
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012,93
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018,93


In [9]:
# Extract macro category
join2 = pd.merge(join, ateco, on='Codice_ateco', how='inner')
join2

,GENERE,ETA,SETTOREECONOMICODETTAGLIO_x,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO


In [10]:
# Delete the sub category
join2 = join2.drop(labels='SETTOREECONOMICODETTAGLIO_x', axis=1)

In [11]:
# Transformations of the strings in upper case, to make a match, without problem
join2['CONTRATTO']=join2['CONTRATTO'].str.upper()
join2

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO


In [12]:
# Select only the indeterminate contracts
indeterminato = join2[join2['CONTRATTO'].str.match(r'.*INDETERMINATO.*')== True].reset_index()
indeterminato=indeterminato.drop(labels=['index', 'CONTRATTO'], axis=1)
indeterminato

,GENERE,ETA,TITOLOSTUDIO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,31,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,SENEGAL,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,M,46,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,BOLIVIA,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,35,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,ECUADOR,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,66,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2008,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,M,45,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...
1353347,F,26,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,BERGAMO,ITALIA,2019,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE
1353348,F,28,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2014,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE
1353349,M,33,LICENZA MEDIA,TEMPO PIENO,MILANO,ITALIA,2009,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE
1353350,M,34,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2021,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE


### Label Encoder

In [13]:
# Starting the procedure to encode the data
titolostudio = indeterminato.TITOLOSTUDIO.tolist()
modlavoro = indeterminato.MODALITALAVORO.tolist()
prov = indeterminato.PROVINCIAIMPRESA.tolist()
nazionalita = indeterminato.ITALIANO.tolist()
copia = indeterminato.copy()

In [14]:
# Instantiate labelencoder object for each column that we want to encode
le_titolostudio = LabelEncoder()
le_modlavoro = LabelEncoder()
le_prov = LabelEncoder()
le_nazionalita = LabelEncoder()

In [15]:
# Apply the label encoder to the dataset
le_titolostudio.fit(copia['TITOLOSTUDIO'])
le_modlavoro.fit(copia['MODALITALAVORO'])
le_prov.fit(copia['PROVINCIAIMPRESA'])
le_nazionalita.fit(copia['ITALIANO'])

LabelEncoder()

In [16]:
# Transform into integers
transformation_ts = le_titolostudio.transform(indeterminato['TITOLOSTUDIO']) 
transformation_ts

array([ 8,  8,  8, ...,  6,  4, 10])

In [17]:
# Try to invert a transformation using the object
list(le_titolostudio.inverse_transform([8]))

['NESSUN TITOLO DI STUDIO']

In [18]:
# Transform into integers
transformation_ml = le_modlavoro.transform(indeterminato['MODALITALAVORO']) 
transformation_prov = le_prov.transform(indeterminato['PROVINCIAIMPRESA']) 
transformation_nazion = le_nazionalita.transform(indeterminato['ITALIANO']) 

In [19]:
# Apply label encoder on categorical feature columns
copia['titolostudio_transformed'] = list(transformation_ts)
copia['modalitalavoro_transformed'] = list(transformation_ml)
copia['provincia_transformed'] = list(transformation_prov)
copia['nazionalita_transformed'] = list(transformation_nazion)
copia

,GENERE,ETA,TITOLOSTUDIO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed
0,F,31,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,SENEGAL,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,141
1,M,46,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,BOLIVIA,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,21
2,F,35,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,ECUADOR,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,50
3,F,66,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2008,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,58
4,M,45,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353347,F,26,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,BERGAMO,ITALIA,2019,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,0,84
1353348,F,28,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2014,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,7,84
1353349,M,33,LICENZA MEDIA,TEMPO PIENO,MILANO,ITALIA,2009,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,6,4,7,84
1353350,M,34,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2021,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,7,84


### Encoding Manually Ordinal Categorical Features

In [20]:
# Create mapper
scale_mapper = {'M':0, 'F':1}

# Map feature values to scale
copia['genere_transformed'] = copia['GENERE'].replace(scale_mapper)
copia

,GENERE,ETA,TITOLOSTUDIO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,F,31,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,SENEGAL,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,141,1
1,M,46,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,BOLIVIA,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,21,0
2,F,35,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,ECUADOR,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,50,1
3,F,66,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2008,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,58,1
4,M,45,NESSUN TITOLO DI STUDIO,TEMPO PARZIALE ORIZZONTALE,LECCO,FILIPPINE,2010,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,8,2,4,58,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353347,F,26,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,BERGAMO,ITALIA,2019,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,0,84,1
1353348,F,28,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2014,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,7,84,1
1353349,M,33,LICENZA MEDIA,TEMPO PIENO,MILANO,ITALIA,2009,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,6,4,7,84,0
1353350,M,34,LAUREA - Vecchio o nuovo ordinamento,TEMPO PIENO,MILANO,ITALIA,2021,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE,4,4,7,84,0


In [46]:
# Trasform the column in int type
copia['Codice_ateco']=copia.Codice_ateco.astype(int)

# We remove the outliers. 
copia = copia[copia['ETA']>=15]
copia = copia[copia['ETA']<100]
copia = copia[copia['ANNO']<2022]
copia

In [21]:
# Drop the columns that have been transformed
transformed_indet = copia.drop(labels =['GENERE', 'TITOLOSTUDIO', 'MODALITALAVORO', 'PROVINCIAIMPRESA', 'ITALIANO', 'SETTOREECONOMICODETTAGLIO_y'], axis=1)
transformed_indet

,ETA,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,31,2010,97,8,2,4,141,1
1,46,2010,97,8,2,4,21,0
2,35,2010,97,8,2,4,50,1
3,66,2008,97,8,2,4,58,1
4,45,2010,97,8,2,4,58,0
...,...,...,...,...,...,...,...,...
1353347,26,2019,06,4,4,0,84,1
1353348,28,2014,06,4,4,7,84,1
1353349,33,2009,06,6,4,7,84,0
1353350,34,2021,06,4,4,7,84,0


In [22]:
# Change the type of all column
transformed_indet['Codice_ateco'] = transformed_indet['Codice_ateco'].astype(str).astype(int)
transformed_indet.dtypes

ETA                           int64
ANNO                          int64
Codice_ateco                  int32
titolostudio_transformed      int32
modalitalavoro_transformed    int32
provincia_transformed         int32
nazionalita_transformed       int32
genere_transformed            int64
dtype: object

In [23]:
transformed_indet

,ETA,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,31,2010,97,8,2,4,141,1
1,46,2010,97,8,2,4,21,0
2,35,2010,97,8,2,4,50,1
3,66,2008,97,8,2,4,58,1
4,45,2010,97,8,2,4,58,0
...,...,...,...,...,...,...,...,...
1353347,26,2019,6,4,4,0,84,1
1353348,28,2014,6,4,4,7,84,1
1353349,33,2009,6,6,4,7,84,0
1353350,34,2021,6,4,4,7,84,0


### Group by features given as input

The idea is to consider non-modifiable features for each person (gender, age, and nationality) and to target an indeterminated contract. 
The result, will be to get the best combination of all other features to get the contract.

In [77]:
# Function to transform the columns ETA, nazionalita_transformed, genere_transformed in a unique column
# with the format: "ETA - nazionalita_transformed - genere_transformed"
# syntax:
# df: Dataframe
# column: column name of the new column
def trasformed_input(df,column):
    df[column] = df["ETA"].apply(str) + " - " + df["genere_transformed"].apply(str)+ " - " + df["nazionalita_transformed"].apply(str)
    return df.drop(labels=['ETA','nazionalita_transformed','genere_transformed'], axis=1)

In [78]:
# Function to retrive the original data from the trasformed ones
# syntax:
# df: Dataframe
# column: column name of the trasformed column
def inverse_trasformed_input(df,column):
    if type(df) is str:
        eta,genere,nazion=df.split(" - ")
        print("età: "+eta+" genere: "+genere+" nazion: "+nazion)
        return eta,genere,nazion
    
    df[['ETA','genere_transformed','nazionalita_transformed']] = df[column].str.split(" - ",expand=True,)
    return df.drop(labels =[column], axis=1)

In [79]:
# Function to retrive the original data from the trasformed ones
# syntax:
# value: trasfomed value
# return 'M' if value is 0 otherwise 'F'
def gender(value):
    if value=='0':
        return 'M'
    else:
        return 'F'

In [80]:
# Encoding of the input
transformed_indet=trasformed_input(transformed_indet,'Input')
transformed_indet

,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,Input
0,2010,97,8,2,4,31 - 1 - 141
1,2010,97,8,2,4,46 - 0 - 21
2,2010,97,8,2,4,35 - 1 - 50
3,2008,97,8,2,4,66 - 1 - 58
4,2010,97,8,2,4,45 - 0 - 58
...,...,...,...,...,...,...
1353347,2019,6,4,4,0,26 - 1 - 84
1353348,2014,6,4,4,7,28 - 1 - 84
1353349,2009,6,6,4,7,33 - 0 - 84
1353350,2021,6,4,4,7,34 - 0 - 84


In [81]:
# Check if the decoding of the input is correct
transformed_indet=inverse_trasformed_input(transformed_indet,'Input')
transformed_indet

,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,ETA,genere_transformed,nazionalita_transformed
0,2010,97,8,2,4,31,1,141
1,2010,97,8,2,4,46,0,21
2,2010,97,8,2,4,35,1,50
3,2008,97,8,2,4,66,1,58
4,2010,97,8,2,4,45,0,58
...,...,...,...,...,...,...,...,...
1353347,2019,6,4,4,0,26,1,84
1353348,2014,6,4,4,7,28,1,84
1353349,2009,6,6,4,7,33,0,84
1353350,2021,6,4,4,7,34,0,84


In [82]:
# Encoding of the input
transformed_indet = trasformed_input(transformed_indet,'Input')
transformed_indet

,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,Input
0,2010,97,8,2,4,31 - 1 - 141
1,2010,97,8,2,4,46 - 0 - 21
2,2010,97,8,2,4,35 - 1 - 50
3,2008,97,8,2,4,66 - 1 - 58
4,2010,97,8,2,4,45 - 0 - 58
...,...,...,...,...,...,...
1353347,2019,6,4,4,0,26 - 1 - 84
1353348,2014,6,4,4,7,28 - 1 - 84
1353349,2009,6,6,4,7,33 - 0 - 84
1353350,2021,6,4,4,7,34 - 0 - 84


In [83]:
# With LabelEncoder we encode the input column
copia = transformed_indet.copy()
le_input = LabelEncoder()
le_input.fit(copia['Input'])
transformation_input = le_input.transform(transformed_indet['Input']) 
copia['Input_transformed'] = list(transformation_input)
copia=copia.drop(labels =['Input'], axis=1)
copia

,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,Input_transformed
0,2010,97,8,2,4,3004
1,2010,97,8,2,4,6226
2,2010,97,8,2,4,4003
3,2008,97,8,2,4,8876
4,2010,97,8,2,4,6058
...,...,...,...,...,...,...
1353347,2019,6,4,4,0,1895
1353348,2014,6,4,4,7,2366
1353349,2009,6,6,4,7,3442
1353350,2021,6,4,4,7,3679
